# TASK 2
for each driver, creates a list of standard routes in that order so that the higher in the list a standard route is, the least the diversion of the driver will be, and 
the output of the program is: 

a file called driver.json that has for each driver, the 5 standard routes routes that if the driver does them, it minimizes the diversion. You can test this by considering as pool of standard routes those that originally the company has and also those that you recommend in the recStandard.json. The file driver.json has the following syntax:
[
	{driver:C, routes:[s10, s20, s2, s6, s10}}, 
	{driver:A, routes:[s1, s2, s22, s61, s102]}, 
….
]


In [176]:
import os
HOME = os.getcwd()
print('HOME: ',HOME)

import time
import math
import json
import random
import pandas as pd
import sys
import lxml
import sklearn as sk
import numpy as np

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import HDBSCAN, DBSCAN
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

from scipy.sparse import csr_matrix, issparse, lil_matrix, coo_matrix

from tqdm import tqdm
from pandarallel import pandarallel

from numba import njit, prange, jit
from numba_progress import ProgressBar

import networkx as nx

HOME:  c:\Users\matti\Desktop\CODE\DataMiningProject23-24


In [177]:
STANDARD_FILENAME = "standard_small_final.json"
ACTUAL_FILENAME = "actual_small_final.json"


K_SHINGLES = 2
ALPHA = 0.7 #TODO: not needed maybe

In [178]:
# load standard and actual data
print("\nReading standard data...")
with open(os.path.join('data',STANDARD_FILENAME)) as f:
    standard = json.load(f)

print("\nReading actual data...")
with open(os.path.join('data', ACTUAL_FILENAME)) as f:
    actual = json.load(f)

# load the data into a dataframe
print("\nCreating standard dataframe...")
dfStandard = pd.DataFrame(standard)
print("\nCreating actual dataframe...")
dfActual = pd.DataFrame(actual)

# print head of the dataframes
print(dfStandard.head())
print(dfActual.head())

# get the unique cities and items of the standard data
cities = []
items = []
drivers = []
longestRoute = 0
shortestRoute = np.inf
maxItemQuantity = 0

standardRefIds = []
for index, s in dfStandard.iterrows():
    #print(s)
    idS = s['id']
    route = s['route']
    standardRefIds.append(int(idS[1]))
    for trip in route:
        cities.append(trip['from']) 
        items.extend(trip['merchandise'].keys())
        maxItemQuantity = max(maxItemQuantity, max(trip['merchandise'].values()))
    if len(route) > 0:
        cities.append(route[-1]['to'])
        
    if len(route) > longestRoute:
        longestRoute = len(route)
        
    if len(route) < shortestRoute:
        shortestRoute = len(route)
print("\nFinished preparing standard data")

actualRefStandardIds = []
for index, s in dfActual.iterrows():
    #print(s)
    idS = s['id']
    route = s['route']
    idStandard = s['sroute']
    drivers.append(s['driver'])
    actualRefStandardIds.append(int(idStandard[1]))
    for trip in route:
        cities.append(trip['from'])
        items.extend(trip['merchandise'].keys())
        maxItemQuantity = max(maxItemQuantity, max(trip['merchandise'].values()))
        
    if len(route) > 0:
        cities.append(route[-1]['to'])
        
    if len(route) > longestRoute:
        longestRoute = len(route)
    
    if len(route) < shortestRoute:
        shortestRoute = len(route)
print("\nFinished preparing actual data")

# find the unique cities and items
uniqueCities = sorted(list(set(cities)))
#uniqueCities.insert(0, 'NULL')          # add NULL city, for padding vectors with different lengths (trips in routes)
uniqueItems = sorted(list(set(items)))
uniqueDrivers = sorted(list(set(drivers)))

if shortestRoute < 2:
    K_SHINGLES = 2

threeShingles = []

for i, c1 in enumerate(uniqueCities):
    for j, c2 in enumerate(uniqueCities):
        if i == j:
            continue
        for k, c3 in enumerate(uniqueCities):
            if j == k or i == k:
                continue
            threeShingles.append([c1, c2, c3])
            
permutations = math.perm(len(uniqueCities), K_SHINGLES)

print("\nUnique cities: ", uniqueCities)
print("Unique items: ", uniqueItems)
print("Unique drivers: ", uniqueDrivers)

standardIds = dfStandard['id'].tolist()
print("standardIds: ", standardIds)

print("\nNumber of cities: ", len(uniqueCities))
print("Number of items: ", len(uniqueItems))

print("\nLongest route: ", longestRoute)
print("Shortest route: ", shortestRoute)

print("\nMax item quantity: ", maxItemQuantity)

print("\nNumber of three-shingles: ", len(threeShingles))

print(f"\n{K_SHINGLES}-shingles: ", math.perm(len(uniqueCities), K_SHINGLES))
print(f"{K_SHINGLES}-shingles: ", math.comb(len(uniqueCities), K_SHINGLES))

print(f"\n\033[92mK-Shingles used: {K_SHINGLES} \033[0m")



Reading standard data...

Reading actual data...

Creating standard dataframe...

Creating actual dataframe...
   id                                              route
0  s0  [{'from': 'Livorno', 'to': 'Catania', 'merchan...
1  s1  [{'from': 'Catania', 'to': 'Altamura', 'mercha...
2  s2  [{'from': 'Barletta', 'to': 'Livorno', 'mercha...
3  s3  [{'from': 'Rho', 'to': 'Caltanissetta', 'merch...
4  s4  [{'from': 'Altamura', 'to': 'Turin', 'merchand...
   id driver sroute                                              route
0  a0      I     s0  [{'from': 'Livorno', 'to': 'Catania', 'merchan...
1  a1      C     s0  [{'from': 'Livorno', 'to': 'Rimini', 'merchand...
2  a2      A     s0  [{'from': 'Livorno', 'to': 'Catania', 'merchan...
3  a3      F     s0  [{'from': 'Livorno', 'to': 'Catania', 'merchan...
4  a4      I     s0  [{'from': 'Livorno', 'to': 'Catania', 'merchan...

Finished preparing standard data

Finished preparing actual data

Unique cities:  ['Altamura', 'Ancona', 'Anzio', 'Asti

In [179]:
def hashShingles(shingles, n):
    # hash shingles
    string = "" 
    for shingle in shingles:
        string += str(shingle) + "," # [45, 4, 8] -> "45,4,8,"
    
    return hash(string) #% n

def createShingles(df, k, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations):
    # create shingles for each route
    shingles = []
    for index, s in df.iterrows():
        idS = s['id']
        route = s['route']
        shingle = [index]
        citiesInRoute = [] # napoli roma milano teramo bergamo [10,4,5,48,12] [10,4,5] [4,5,48] [5,48,12]
        merchandiseInRoute = np.zeros(len(uniqueItems))
        for trip in route:
            citiesInRoute.append(uniqueCities.index(trip['from']))
            #merchandiseInRoute += np.array(list(trip['merchandise'].values()))
            for item, n in trip['merchandise'].items():
                merchandiseInRoute[uniqueItems.index(item)] += n
        if len(route) > 0:
            citiesInRoute.append(uniqueCities.index(route[-1]['to']))
        if len(route) > 0:
            merchandiseInRoute = merchandiseInRoute / (maxItemQuantity*len(route))
        
        hashedShingles = []
        for i in range(len(citiesInRoute)-k+1):
            # Q: is it correct to set the modulo for the hash function to the number of permutations?
            # A: yes, because we want to have a unique hash for each shingle
            # Q: would it be better to use a different hash function?
            # A: yes, because the modulo function is not a good hash function
            hashedShingles.append(hashShingles(citiesInRoute[i:i+k], permutations) )
        
        shingle.append(np.array(hashedShingles))
        
        shingle.append(merchandiseInRoute) # quantity hot encoding
        
        shingles.append(shingle)
        
    return shingles # [ index, [shingles], [merchandise] ]

def create_shingles(s, k, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations):
    idS = s['id']
    route = s['route']
    shingle = [s.name]
    citiesInRoute = [] 
    merchandiseInRoute = np.zeros(len(uniqueItems))
    for trip in route:
        citiesInRoute.append(uniqueCities.index(trip['from']))
        for item, n in trip['merchandise'].items():
            merchandiseInRoute[uniqueItems.index(item)] += n
    if len(route) > 0:
        citiesInRoute.append(uniqueCities.index(route[-1]['to']))
    if len(route) > 0:
        merchandiseInRoute = merchandiseInRoute / (maxItemQuantity*len(route))
    
    hashedShingles = []
    for i in range(len(citiesInRoute)-k+1):
        hashedShingles.append(hashShingles(citiesInRoute[i:i+k], permutations))
    
    shingle.append(np.array(hashedShingles))
    shingle.append(merchandiseInRoute)
    
    return shingle

In [180]:
def create_shingles_selfcontained(s, k, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations):
        import numpy as np
        def hash_shingles(shingles):
            # hash shingles
            string = ""
            for shingle in shingles:
                string += str(shingle) + ","
            return hash(string)

        idS = s['id']
        route = s['route']
        shingle = [s.name]
        citiesInRoute = []
        merchandiseInRoute = np.zeros(len(uniqueItems))

        for trip in route:
            citiesInRoute.append(uniqueCities.index(trip['from']))
            for item, n in trip['merchandise'].items():
                merchandiseInRoute[uniqueItems.index(item)] += n

        if len(route) > 0:
            citiesInRoute.append(uniqueCities.index(route[-1]['to']))

        if len(route) > 0:
            merchandiseInRoute = merchandiseInRoute / (maxItemQuantity * len(route))

        hashedShingles = []

        for i in range(len(citiesInRoute) - k + 1):
            hashedShingles.append(hash_shingles(citiesInRoute[i:i + k]))

        shingle.append(np.array(hashedShingles))
        shingle.append(merchandiseInRoute)
        return shingle

In [181]:
standardSets = createShingles(dfStandard, k=K_SHINGLES, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)
actualSets = createShingles(dfActual, k=K_SHINGLES, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)

# pandarallel.initialize(progress_bar=True)

# standardSets = dfStandard.parallel_apply(lambda s: create_shingles_selfcontained(s, K_SHINGLES, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations), axis=1)
# standardSets = standardSets.tolist()
# actualSets = dfActual.parallel_apply(lambda s: create_shingles_selfcontained(s, K_SHINGLES, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations), axis=1)
# actualSets = actualSets.tolist()

print("\nstandardSets", len(standardSets), "shape first element", standardSets[0][1].shape, standardSets[0])
print("\nactualSets", len(actualSets),  "shape first element", standardSets[0][1].shape, actualSets[0])

print("\nstandardSets:", len(standardSets))
print("actualSets:", len(actualSets))

assert len(standardSets[0]) == 3, "The length of the standard set is not equal to 3 (index, shingles, merchandise)"
assert len(standardSets[0][2]) == len(uniqueItems), "The length of the merchandise vector is not equal to the number of unique items"


standardSets 10 shape first element (7,) [0, array([  927601937676351067,  7616274575629296849,  5037646654388308800,
        -999797332344628461,  1553328499977496622,  5100688397118852643,
       -7541361884513732122], dtype=int64), array([0.42428571, 0.31142857, 0.29285714, 0.31857143, 0.31142857,
       0.38428571, 0.32857143, 0.29571429, 0.28428571, 0.24714286,
       0.26857143, 0.33428571, 0.27      , 0.24428571, 0.34571429,
       0.25714286, 0.15      , 0.46428571, 0.43285714, 0.25285714,
       0.37285714, 0.36571429, 0.22857143, 0.06142857, 0.20142857,
       0.25285714, 0.55428571, 0.40714286, 0.26571429, 0.46714286])]

actualSets 100 shape first element (7,) [0, array([  927601937676351067,  4297192775114874019,   389033426465705138,
        5037646654388308800,  -999797332344628461,   411325410248094515,
       -7541361884513732122], dtype=int64), array([0.29571429, 0.28571429, 0.20285714, 0.15142857, 0.09      ,
       0.34285714, 0.41285714, 0.37571429, 0.23      , 0.2

### FUNCTIONS

In [182]:
def jaccard_similarity_matrix(matrix):
    intersection = np.dot(matrix, matrix.T)
    row_sums = matrix.sum(axis=1)
    union = row_sums[:, None] + row_sums - intersection
    union = np.where(union == 0, 1, union)  # avoid division by zero
    jaccard_similarity = intersection / union
    return jaccard_similarity

def jaccard_similarity_two_matrices(matrix1, matrix2):
    #intersection = np.dot(matrix, matrix.T)
    intersection = np.dot(matrix1, matrix2.T)
    row_sums1 = matrix1.sum(axis=1)
    row_sums2 = matrix2.sum(axis=1)
    union = row_sums1[:, None] + row_sums2 - intersection
    union = np.where(union == 0, 1, union)  # avoid division by zero
    jaccard_similarity = intersection / union
    return jaccard_similarity

In [183]:
import numpy as np
from scipy.sparse import csr_matrix

def jaccard_similarity_sparse(matrix1, matrix2):
    # Convert dense matrices to sparse matrices (CSR format)
    sparse_matrix1 = csr_matrix(matrix1)
    sparse_matrix2 = csr_matrix(matrix2)

    # Matrix multiplication in CSR format
    intersection = sparse_matrix1.dot(sparse_matrix2.T).toarray()

    # Row sums using CSR format
    row_sums1 = sparse_matrix1.sum(axis=1).A.ravel()
    row_sums2 = sparse_matrix2.sum(axis=1).A.ravel()

    # Calculate union using the correct formula
    union = row_sums1 + row_sums2 - intersection

    # Avoid division by zero
    union = np.where(union == 0, 1, union)

    # Jaccard similarity
    jaccard_similarity = intersection / union
    return jaccard_similarity


In [184]:
def jaccard_similarity_matrix_merch(matrix):
    print("matrix", matrix.shape)
    min_matrix = np.minimum(matrix[:, None, :], matrix[None, :, :])
    sum_min_matrix = np.sum(min_matrix, axis=-1)
    print("sum_min_matrix", sum_min_matrix.shape)
    
    max_matrix = np.maximum(matrix[:, None, :], matrix[None, :, :])
    sum_max_matrix = np.sum(max_matrix, axis=-1)
    print("sum_max_matrix", sum_max_matrix.shape)
    
    jaccard_similarity = sum_min_matrix / sum_max_matrix
    return jaccard_similarity

def jaccard_similarity_matrices_merch(matrix1, matrix2):
    print("matrix1", matrix1.shape)
    print("matrix2", matrix2.shape)
    
    min_matrix = np.minimum(matrix1[:, None, :], matrix2[None, :, :])
    sum_min_matrix = np.sum(min_matrix, axis=-1)
    print("sum_min_matrix", sum_min_matrix.shape)
    
    max_matrix = np.maximum(matrix1[:, None, :], matrix2[None, :, :])
    sum_max_matrix = np.sum(max_matrix, axis=-1)
    print("sum_max_matrix", sum_max_matrix.shape)
    
    jaccard_similarity = sum_min_matrix / sum_max_matrix
    return jaccard_similarity


def create_binary_matrix(routeSets):
    # create binary matrix where each row represents a route
    uniqueShingles = list(set([shingle for route in routeSets for shingle in route[1]]))
    binaryMatrix = np.zeros((len(routeSets), len(uniqueShingles)))
    for i, route in enumerate(routeSets):
        for shingle in route[1]:
            binaryMatrix[i][uniqueShingles.index(shingle)] = 1
    return binaryMatrix



In [185]:
def hash_function_hash_code(num_of_hashes,n_col,next_prime):
  
    #coeffA = np.array(pick_random_coefficients(num_of_hashes,max_column_length)).reshape((num_of_hashes,1))
    #coeffB = np.array(pick_random_coefficients(num_of_hashes,max_column_length)).reshape((num_of_hashes,1))

    coeffA = np.array(random.sample(range(0,n_col*100),num_of_hashes)).reshape((num_of_hashes,1))
    coeffB = np.array(random.sample(range(0,n_col*100),num_of_hashes)).reshape((num_of_hashes,1))

    x = np.arange(n_col).reshape((1,n_col))

    hash_code = (np.matmul(coeffA,x) + coeffB) % next_prime # (num_of_hashes,n_col) so how each column index is permuted

    return hash_code

def minhash(u,num_of_hashes):
    (n_row, n_col) = u.shape
    next_prime = n_col
    hash_code = hash_function_hash_code(num_of_hashes,n_col,next_prime)

    signature_array = np.empty(shape = (n_row,num_of_hashes))

    #t2 = time.time()
    for row in tqdm(range(n_row), desc="minhashing"):
        #print("row", row)
        ones_index = np.where(u[row,:]==1)[0]
        #if len(ones_index) == 0:
        signature_array[row,:] = np.zeros((1,num_of_hashes))
            #continue
        corresponding_hashes = hash_code[:,ones_index]
        #print("ones_index", ones_index.shape, ones_index)
        #print("corresponding_hashes", corresponding_hashes.shape, corresponding_hashes)
        row_signature = np.amin(corresponding_hashes,axis=1).reshape((1,num_of_hashes))

        signature_array[row,:] = row_signature

    return signature_array

## NEW FUNCTIONS FOR TASK 2

In [186]:
def create_binary_matrices(routeSet1, routeSet2):
    # create binary matrix where each row represents a route
    uniqueShinglesBoth = list(set([shingle for route in routeSet1 for shingle in route[1]] + [shingle for route in routeSet2 for shingle in route[1]]))
    binaryMatrix1 = np.zeros((len(routeSet1), len(uniqueShinglesBoth)))
    binaryMatrix2 = np.zeros((len(routeSet2), len(uniqueShinglesBoth)))
    for i, route in enumerate(routeSet1):
        for shingle in route[1]:
            binaryMatrix1[i][uniqueShinglesBoth.index(shingle)] = 1
            
    for i, route in enumerate(routeSet2):
        for shingle in route[1]:
            binaryMatrix2[i][uniqueShinglesBoth.index(shingle)] = 1
    return binaryMatrix1, binaryMatrix2

def find_num_hashes_minhash(matrix):
    if matrix.shape[1]<150:
        num_hash_functions = matrix.shape[1]
    elif matrix.shape[1]<500:
        num_hash_functions = matrix.shape[1]//2
    elif matrix.shape[1] < 1000:
        num_hash_functions = matrix.shape[1]//10
    elif matrix.shape[1] < 10_000:
        num_hash_functions = 150
    elif matrix.shape[1] < 100_000:
        num_hash_functions = 250
    else:
        num_hash_functions = 300
    return num_hash_functions

In [187]:
def hash_function_hash_code(num_of_hashes,n_col,next_prime):

    coeffA = np.array(random.sample(range(0,n_col*100),num_of_hashes)).reshape((num_of_hashes,1))
    coeffB = np.array(random.sample(range(0,n_col*100),num_of_hashes)).reshape((num_of_hashes,1))

    x = np.arange(n_col).reshape((1,n_col))

    hash_code = (np.matmul(coeffA,x) + coeffB) % next_prime # (num_of_hashes,n_col) so how each column index is permuted

    return hash_code

# def minhash(u,num_of_hashes):
#     (n_row, n_col) = u.shape
#     next_prime = n_col
#     hash_code = hash_function_hash_code(num_of_hashes,n_col,next_prime)

#     signature_array = np.empty(shape = (n_row,num_of_hashes))

#     #t2 = time.time()

#     for row in tqdm(range(n_row), desc="minhashing"):
#         #print("row", row)
#         ones_index = np.where(u[row,:]==1)[0]
#         #if len(ones_index) == 0:
#         signature_array[row,:] = np.zeros((1,num_of_hashes))
#             #continue
#         corresponding_hashes = hash_code[:,ones_index]
#         #print("ones_index", ones_index.shape, ones_index)
#         #print("corresponding_hashes", corresponding_hashes.shape, corresponding_hashes)
#         row_signature = np.amin(corresponding_hashes,axis=1).reshape((1,num_of_hashes))

#         signature_array[row,:] = row_signature

#     return signature_array

def minhash_matrices(matrix1,matrix2,num_of_hashes):
    (n_row, n_col) = matrix1.shape
    next_prime = n_col
    hash_code = hash_function_hash_code(num_of_hashes,n_col,next_prime)

    signature1_array = np.empty(shape = (n_row,num_of_hashes))
    signature2_array = np.empty(shape = (matrix2.shape[0],num_of_hashes))

    #t2 = time.time()

    for row in tqdm(range(n_row), desc="minhashing"):
        #print("row", row)
        ones_index = np.where(matrix1[row,:]==1)[0]
        #if len(ones_index) == 0:
        signature1_array[row,:] = np.zeros((1,num_of_hashes))

            #continue
        corresponding_hashes = hash_code[:,ones_index]
        #print("ones_index", ones_index.shape, ones_index)
        #print("corresponding_hashes", corresponding_hashes.shape, corresponding_hashes)
        row_signature = np.amin(corresponding_hashes,axis=1).reshape((1,num_of_hashes))

        signature1_array[row,:] = row_signature

    for row in tqdm(range(matrix2.shape[0]), desc="minhashing"):
        #print("row", row)
        ones_index = np.where(matrix2[row,:]==1)[0]
        #if len(ones_index) == 0:
        signature2_array[row,:] = np.zeros((1,num_of_hashes))

            #continue
        corresponding_hashes = hash_code[:,ones_index]
        #print("ones_index", ones_index.shape, ones_index)
        #print("corresponding_hashes", corresponding_hashes.shape, corresponding_hashes)
        row_signature = np.amin(corresponding_hashes,axis=1).reshape((1,num_of_hashes))

        signature2_array[row,:] = row_signature

    return signature1_array, signature2_array

def find_band_and_row_values(columns, threshold):
    previous_b = 1
    previous_r = columns
    for b in range(1, columns + 1):
        if columns % b == 0:
            r = columns // b
            if (1 / b) ** (1 / r)  <= threshold:
                if np.abs((1 / previous_b) ** (1 / previous_r) - threshold) < np.abs((1 / b) ** (1 / r) - threshold):
                    return previous_b, previous_r
                return b, r
    return columns, 1

In [188]:
def jaccard_similarity_minhash_lsh_route_merch(matrix, matrixMerch, thresh_user=0.2):
    #similarity_matrix = csr_matrix((matrix.shape[0], matrix.shape[0]), dtype=np.float64)
    #similarity_matrix = lil_matrix((matrix.shape[0], matrix.shape[0]), dtype=np.float64)
    pairs = lsh(matrix, thresh_user=thresh_user)
    #uniqueRows = np.unique([i for i, j in pairs] + [j for i, j in pairs])
    uniqueRowsSet = set([i for i, j in pairs] + [j for i, j in pairs]) # (1,2) (1,4) (1,5)
    neverSeen = set([i for i in range(matrix.shape[0])]) - uniqueRowsSet
    print("neverSeen", neverSeen)
    #print("uniqueRows numpy", len(uniqueRows))
    print("num of subset of rows to check similarity:", len(uniqueRowsSet))
    #print(" num of pairs", len(uniqueRowsSet)*(len(uniqueRowsSet)-1)/2)
    print(" num of pairs", len(pairs))
    print(" instead of", matrix.shape[0]*(matrix.shape[0]-1)/2)
    print("improved by", (1 - len(pairs) / (matrix.shape[0]*(matrix.shape[0]-1)/2)) *100, "%")
    
        
    print("Computing jaccard similarity on subset matrix...")
    #print("subset matrix", subset_matrix.shape)

    # with ProgressBar(total=len(pairs)) as progress:
    #     distance_pairs = compute_subset_similarity_matrix_only_pairs(matrix, matrixMerch, pairs, progress)
    
    sortedUniqueRowsSet = sorted(list(uniqueRowsSet))
    subset_matrix = matrix[sortedUniqueRowsSet]
    subset_matrixMerch = matrixMerch[sortedUniqueRowsSet]
    print("subset_matrix", subset_matrix.shape, subset_matrix[0])
    print("subset_matrixMerch", subset_matrixMerch.shape, subset_matrixMerch[0])
    with ProgressBar(total=len(sortedUniqueRowsSet)) as progress:
        subset_sim_matrix = compute_subset_similarity_matrix_and_merch(subset_matrix, subset_matrixMerch, progress)
    print("subset_sim_matrix", subset_sim_matrix.shape, subset_sim_matrix[0])
    print("subset_sim_matrix contains nan", np.isnan(subset_sim_matrix).any())
    print("nan indices", len(np.argwhere(np.isnan(subset_sim_matrix))), np.argwhere(np.isnan(subset_sim_matrix)))
    
    # if len(neverSeen) > 0:
    #     for i, n in enumerate(neverSeen):
    #         distance_pairs = np.concatenate([distance_pairs, [1]*(matrix.shape[0]-1-i)])
        
    #     pairs = np.concatenate([pairs, np.array([[i, j] for i,n  in enumerate(neverSeen) for j in range(i, matrix.shape[0]) if i != j])])
    #print("pairs", pairs.shape, pairs[-10:])
    # map back to original matrix
    print("Mapping back to original matrix...")
    
    lenMatrixNoNeverSeen = matrix.shape[0] - len(neverSeen)
    
    # remove never seen rows and map indices
    map_indices = {}
    sortedNeverSeen = sorted(list(neverSeen))
    counter = 0
    for i in range(matrix.shape[0]):
        if i in sortedNeverSeen:
            continue
        map_indices[i] = counter
        counter += 1
        
    print("map_indices", map_indices)
    map_indices_back = {v: k for k, v in map_indices.items()}
    
  
    subset_sim_matrix = csr_matrix(subset_sim_matrix)
    
    return subset_sim_matrix, map_indices_back

In [189]:
def lsh(minhash_matrix, thresh_user=0.2):
    # Initialize the signature matrix
    columns = minhash_matrix.shape[1]
    
    # Generate the hash functions
   # hash_functions = [lambda x, a=a, b=b: (a * x + b) % minhash_matrix.shape[1] for a, b in zip(random.sample(range(1000), bands), random.sample(range(1000), bands))]
    hash_function = lambda x: hash(",".join([str(x[i]) for i in range(len(x))]))
    
    # b = bands
    # r = columns//bands
    b, r = find_band_and_row_values(columns, thresh_user)
    # If columns is not divisible by bands
    if columns % b != 0:
        # Find the closest number that makes it divisible
        while columns % b != 0:
            b -= 1
        r = columns // b
    #bands = b
        
    print("final bands", b)
    signature_matrix = np.full((minhash_matrix.shape[0], b), np.inf)
    
    # if threshold is 0.8,
    threshold = (1 / b) ** (1 / r) 
    print("lsh threshold", threshold)
    
    # For each band
    print("Computing hash values of bands...")
    hash_values = np.apply_along_axis(lambda x: hash_function(x) % minhash_matrix.shape[0], 1, minhash_matrix.reshape(-1, r))
    # Reshape the hash values to match the signature matrix
    hash_values = hash_values.reshape(minhash_matrix.shape[0], b)
    # Update the signature matrix
    signature_matrix = hash_values
            
    # find candidate pairs
    print("Finding candidate pairs...")
    candidate_pairs = []
    for i in tqdm(range(signature_matrix.shape[0])):
        # Compute the similarity of the current row with all following rows
        similarities = np.sum(signature_matrix[i+1:, :] == signature_matrix[i, :], axis=1) / b
        # Find the indices of the rows that have a similarity greater than or equal to the threshold
        indices = np.nonzero(similarities >= threshold)[0]
        # Add the pairs to the candidate pairs
        candidate_pairs.extend((i, i+1+index) for index in indices)
    
    return np.array(candidate_pairs)

def lsh_two_matrices(minhash_matrix1, minhash_matrix2, thresh_user=0.2):
    # Initialize the signature matrix
    columns = minhash_matrix1.shape[1]
    
    # Generate the hash functions
    # hash_functions = [lambda x, a=a, b=b: (a * x + b) % minhash_matrix.shape[1] for a, b in zip(random.sample(range(1000), bands), random.sample(range(1000), bands))]
    # hash_function = lambda x: hash(",".join([str(x[i]) for i in range(len(x))]))
    
    def hash_function(x):
        # print("x",x)
        var = hash(",".join([str(x[i]) for i in range(len(x))]))
        # print ("str x ", (",".join([(x[i]) for i in range(len(x))])))
        # print ("var", var)
        return var % minhash_matrix1.shape[0]


    # b = bands
    # r = columns//bands
    b, r = find_band_and_row_values(columns, thresh_user)
    # If columns is not divisible by bands
    if columns % b != 0:
        # Find the closest number that makes it divisible
        while columns % b != 0:
            b -= 1
        r = columns // b
        
    print("final bands", b)
    signature_matrix1 = np.full((minhash_matrix1.shape[0], b), np.inf)
    signature_matrix2 = np.full((minhash_matrix2.shape[0], b), np.inf)
    

    threshold = (1 / b) ** (1 / r) 
    print("lsh threshold", threshold)
    
    # # For each band
    # print("Computing hash values of bands...")
    # hash_values1 = np.apply_along_axis(lambda x: hash_function(x) % minhash_matrix1.shape[0], 1, minhash_matrix1.reshape(-1, r))
    # print("hash_values1", hash_values1.shape, hash_values1)
    # hash_values2 = np.apply_along_axis(lambda x: hash_function(x) % minhash_matrix2.shape[0], 1, minhash_matrix2.reshape(-1, r))
    # print("hash_values2", hash_values2.shape, hash_values2)

    print("minhash_matrix1.reshape(-1, r).shape",minhash_matrix1.reshape(-1, r).shape)

    # For each band
    print("Computing hash values of bands...")
    hash_values1 = np.apply_along_axis(hash_function, 1, minhash_matrix1.reshape(-1, r))
    # print("hash_values1", hash_values1.shape, hash_values1)
    hash_values2 = np.apply_along_axis(hash_function, 1, minhash_matrix2.reshape(-1, r))
    # print("hash_values2", hash_values2.shape, hash_values2)


    # Reshape the hash values to match the signature matrix
    hash_values1 = hash_values1.reshape(minhash_matrix1.shape[0], b)
    # print("hash_values1", hash_values1.shape, hash_values1)
    hash_values2 = hash_values2.reshape(minhash_matrix2.shape[0], b)
    # print("hash_values2", hash_values2.shape, hash_values2) 
    # Update the signature matrix
    signature_matrix1 = hash_values1
    signature_matrix2 = hash_values2
    
    
    # find candidate pairs
    print("Finding candidate pairs...")
    # similarities_actual=[]
    # candidate_pairs = np.empty((minhash_matrix1.shape[0], 2))

    data=[]
    rows=[]
    cols=[]

    for i in tqdm(range(signature_matrix1.shape[0])):
        # Compute the similarity of the current row with all following rows
        similarities = np.sum(signature_matrix2 == signature_matrix1[i, :], axis=1) / b
        # print("similarities", similarities.shape, similarities)
        # Find the indices of the rows that have a similarity greater than or equal to the threshold
        indices = np.nonzero(similarities >= threshold)[0]
        # print("indices", indices.shape, indices)

        # print("similarities[indices] ",similarities[indices])

        data.extend(similarities[indices])
        # print("data", data)
        rows.extend([i]*len(indices))
        # print("rows", rows)
        cols.extend(indices)
        # print("cols", cols)
        # indexMax = np.argmax(similarities)
        # simMax = similarities[indexMax]
        # # Add the pairs to the candidate pairs
        # #candidate_pairs.extend((i, i+1+index) for index in indices)
        # candidate_pairs[i] = [indexMax, simMax]
        # similarities_actual.append(similarities)

        

    # # Create data array for COO matrix
    # data = np.concatenate([subset_sim_matrix[indices_i, indices_j], subset_sim_matrix[indices_i, indices_j]])
    
    # # Create row and column index arrays for COO matrix
    # rows = np.concatenate([indices_i_mapped, indices_j_mapped])
    # cols = np.concatenate([indices_j_mapped, indices_i_mapped])
    # print("data", data)
    # print("rows", rows)
    # print("cols", cols)

    similarity_matrix = coo_matrix((data, (rows, cols)), shape=(minhash_matrix1.shape[0], minhash_matrix2.shape[0])).tocsr()

    return similarity_matrix





In [190]:
a = np.array([[1],[2],[3]])
b = np.array([[1],[2],[2]])
print("a==b", a[:]==b[:])
similarities = np.sum(b == a, axis=1)/3
print("similarities", similarities)
threshold = 0.1
indices = np.nonzero(similarities >= threshold)[0]
print("indices", indices)

a==b [[ True]
 [ True]
 [False]]
similarities [0.33333333 0.33333333 0.        ]
indices [0 1]


In [191]:
@jit(cache=True, nogil=True, parallel=True)
def compute_distance_pairs_Merch(sim_matrix, matrix1, matrix1Merch, matrix2, matrix2Merch, progress_proxy):
    n = sim_matrix.shape[0]
    m = sim_matrix.shape[1]
    
    # print("sim_matrix", sim_matrix.shape)    
    # print(numba.typeof(sim_matrix))
    # print(numba.typeof(matrix1))
    # print(numba.typeof(matrix1Merch))
    # print(numba.typeof(matrix2))
    # print(numba.typeof(matrix2Merch))
    # print(numba.typeof(progress_proxy))


    for i in prange(n):
        subset1 = matrix1[i].reshape(1, -1) #replicate_row(subset_matrix, i) 
        # print("subset1", subset1.shape)
        subset2 = matrix2[sim_matrix[i].nonzero()[1]]
        # print("subset2", subset2.shape)
        min_matrix = np.minimum(subset1, subset2)
        sum_min_matrix = np.sum(min_matrix, axis=-1)
        
        max_matrix = np.maximum(subset1, subset2)
        sum_max_matrix = np.sum(max_matrix, axis=-1)

        route_distance = (np.divide(sum_min_matrix, sum_max_matrix))
        # print("route_distance", route_distance.shape)

        subset1Merch = matrix1Merch[i].reshape(1, -1) #replicate_row(subset_matrixMerch, i)
        subset2Merch = matrix2Merch[sim_matrix[i].nonzero()[1]]
        # print("subset1Merch", subset1Merch.shape)
        # print("subset2Merch", subset2Merch.shape)

        min_matrixMerch = np.minimum(subset1Merch, subset2Merch)
        sum_min_matrixMerch = np.sum(min_matrixMerch, axis=-1)

        max_matrixMerch = np.maximum(subset1Merch, subset2Merch)
        sum_max_matrixMerch = np.sum(max_matrixMerch, axis=-1)

        merch_distance = (np.divide(sum_min_matrixMerch, sum_max_matrixMerch))
        # print("merch_distance", merch_distance.shape)



        sim_matrix[i,sim_matrix[i].nonzero()[1]] = (0.8) * route_distance + (0.2) * merch_distance

        progress_proxy.update(1)
    
    return sim_matrix


def similarity_minhash_lsh_two_matrices(matrix1, matrix1Merch, matrix2, matrix2Merch, thresh_user=0.2):
    
    similarity_matrix = lsh_two_matrices(matrix1,matrix2, thresh_user=thresh_user)
    # print("similarity_matrix", similarity_matrix.shape, similarity_matrix[0])
    # print("similarity_matrix", similarity_matrix.shape, similarity_matrix[1])
    # print("similarity_matrix", similarity_matrix.shape, similarity_matrix[101])
    # print("similarity_matrix[0]", similarity_matrix[0,0])
    # print("similarity_matrix[9,9]", similarity_matrix[9,9])

    # uniqueRowsSet = set([i for i, j in pairs] + [j for i, j in pairs]) # (1,2) (1,4) (1,5)
    # neverSeen = set([i for i in range(matrix1.shape[0])]) - uniqueRowsSet
    

    # sortedUniqueRowsSet = sorted(list(uniqueRowsSet))
    # print("sortedUniqueRowsSet", sortedUniqueRowsSet)

    # subset_matrix1 = matrix1[sortedUniqueRowsSet]
    # subset_matrix1Merch = matrix1Merch[sortedUniqueRowsSet]
    # print("subset_matrix1", subset_matrix1.shape, subset_matrix1[0])
    # print("subset_matrix1Merch", subset_matrix1Merch.shape, subset_matrix1Merch[0])

    # subset_matrix2 = matrix2[sortedUniqueRowsSet]
    # subset_matrix2Merch = matrix2Merch[sortedUniqueRowsSet]
    # print("subset_matrix2", subset_matrix2.shape, subset_matrix2[0])
    # print("subset_matrix2Merch", subset_matrix2Merch.shape, subset_matrix2Merch[0])

    # subset_similarity_matrix = np.full((subset_matrix1.shape[0], subset_matrix2.shape[0]), np.inf)
        
    print("Computing distance  on subset matrix...")
    with ProgressBar(total=matrix1.shape[0]) as progress:
        similarity_matrix = compute_distance_pairs_Merch(similarity_matrix, matrix1, matrix1Merch, matrix2, matrix2Merch, progress)
    
    print("similarity_matrix", similarity_matrix.shape, similarity_matrix[0])
    print("similarity_matrix", similarity_matrix.shape, similarity_matrix[1])
    # print("similarity_matrix", similarity_matrix.shape, similarity_matrix[101])

    return similarity_matrix

C:\Users\matti\AppData\Local\Temp/ipykernel_10356/4185096283.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(cache=True, nogil=True, parallel=True)


## task 2

In [192]:
# convert routes and merchandise to binary matrices

print("Creating route binary matrix...")
route_matrix, route_matrix_standard = create_binary_matrices(actualSets, standardSets)
print("\nroute_matrix actual", route_matrix.shape, route_matrix[0])
print("\nroute_matrix standard", route_matrix_standard.shape, route_matrix_standard[0])
num_hash_functions = find_num_hashes_minhash(route_matrix)

print("Minhashing route matrix...")    
route_matrix, route_matrix_standard = minhash_matrices(route_matrix, route_matrix_standard, num_hash_functions if num_hash_functions % 2 == 0 else num_hash_functions + 1)
print("\nroute_matrix minhash", route_matrix.shape, route_matrix[0])
print("\nroute_matrix_standard minhash", route_matrix_standard.shape, route_matrix_standard[0])


print("\nACTUAL")

print("\nCreating merchandise binary matrix...")
merch_matrix = np.array([s[2] for s in actualSets])
print("\nmerch_matrix", merch_matrix.shape, merch_matrix[0])
print("merch_matrix contains nan", np.isnan(merch_matrix).any())

print("\nSTANDARD")

print("\nCreating merchandise binary matrix...")
merch_matrix_standard = np.array([s[2] for s in standardSets])
print("\nmerch_matrix_standard", merch_matrix.shape, merch_matrix[0])
print("merch_matrix_standard contains nan", np.isnan(merch_matrix).any())

# Essentials for Task 2
# standardToActualSetsDistances = None
print("Computing distance route matrix actual to standard...")

# route_matrix_distance_actual_standard, map_indices_back = distance_minhash_lsh_two_matrices(route_matrix, merch_matrix, route_matrix_standard, merch_matrix_standard, thresh_user=0.0)
# print("\nroute_similarity_standard_to_actual", route_similarity_standard_to_actual.shape, route_similarity_standard_to_actual[0])

# merch_similarity_lsh_standard_to_actual = jaccard_similarity_minhash_lsh_two_matrices(merch_matrix, merch_matrix_standard, thresh_user=0.0)



Creating route binary matrix...

route_matrix actual (100, 591) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

minhashing: 100%|██████████| 100/100 [00:00<00:00, 33399.46it/s]


minhashing: 100%|██████████| 10/10 [00:00<00:00, 10043.83it/s]


route_matrix minhash (100, 60) [ 44.  62.   1.  27.   0.  47.  26. 175.  22.   9.  86.  25.  47.  31.
  70.  39.  31. 276.  31. 124.  24.  25.  15. 179. 103. 127. 150.  92.
  74.  49.   2.  60. 180. 204.  96. 136.  39.  42.  12.  73.   0. 129.
 185.  52.  83.   3.  84.  46.  25.  15.  40.  12.  32.  15.  21.  16.
   3.  74. 179. 147.]

route_matrix_standard minhash (10, 60) [  2.  54.   1.  27.   0.  59.  37. 175.  68.   9.  86.  25. 425.  31.
  70.  39.  25. 152.  31.   6.  28.  25.  15. 130.  24.  81. 150.  86.
  13.  14. 116.  60.  94.  80.  96. 121.  26.  42.  12.  22. 189.   2.
  18.  52.  58. 175.  91.  21.  25.  15.  60.  12.  32.  27.  21.  16.
  64.  74.  30. 147.]

ACTUAL

Creating merchandise binary matrix...

merch_matrix (100, 30) [0.29571429 0.28571429 0.20285714 0.15142857 0.09       0.34285714
 0.41285714 0.37571429 0.23       0.24714286 0.22       0.27
 0.16       0.21142857 0.44285714 0.24       0.10285714 0.33142857
 0.24       0.15714286 0.29142857 0.22142857 0.198

In [193]:
route_matrix_similarity_actual_standard = similarity_minhash_lsh_two_matrices(route_matrix, merch_matrix, route_matrix_standard, merch_matrix_standard, thresh_user=0.3)

final bands 30
lsh threshold 0.18257418583505536
minhash_matrix1.reshape(-1, r).shape (3000, 2)
Computing hash values of bands...
Finding candidate pairs...


100%|██████████| 100/100 [00:00<00:00, 50249.24it/s]

Computing distance  on subset matrix...


  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\matti\AppData\Local\Temp/ipykernel_10356/4185096283.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "compute_distance_pairs_Merch" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\matti\AppData\Local\Temp/ipykernel_10356/4185096283.py (1)

File "..\..\..\AppData\Local\Temp\ipykernel_10356\4185096283.py", line 1:
<source missing, REPL/exec in use?>

  @jit(cache=True, nogil=True, parallel=True)
C:\Users\matti\AppData\Local\Temp/ipykernel_10356/4185096283.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "compute_distance_pairs_Merch" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\AppData\Local\Temp\ipykernel_10356\4185096283.py", line 15:
<source missing, REPL/exec in use?>

  @jit(cache=True, nogil=True, parallel=True)
c:\Users\matti\anaconda3\e

similarity_matrix (100, 10) 
similarity_matrix (100, 10) 


In [194]:
print("\nroute_matrix_distance_actual_standard", route_matrix_similarity_actual_standard[99:101])

max_value = np.max(route_matrix_similarity_actual_standard, axis=1).toarray()
# print("max_value", max_value.shape, max_value[0:12])

max_value_index = np.argmax(route_matrix_similarity_actual_standard, axis=1)
# print("max_value_index", max_value_index.shape, max_value_index[0:12])

# Set max_value_index to -1 where max_value is zero
max_value_index[max_value == 0] = -1

# print("max_value_index", max_value_index.shape, max_value_index[0:12])



route_matrix_distance_actual_standard 


In [195]:
# print(route_matrix_similarity_actual_standard)
# print(max_value_index)

In [196]:
# for i, idx in enumerate(max_value_index):
#     print(i, " --> " ,idx)
#     if (i+1)%10==0:
#         print("-----")

In [197]:
# print("Index of the biggest value for each row:", max_value_index[850])
# print("Value of the biggest value for each row:", max_value[850])


In [198]:

print("len(max_value_index)", len(max_value_index))
print("len(max_value)", len(max_value))

# [index for index, s in dfActual.iterrows() if s['driver'] == driver]
# uniqueDrivers
driver_indices = {}

for i, s in dfActual.iterrows():
    driver = s['driver']
    
    # Check if the driver is already in the dictionary
    if driver in driver_indices:
        # If yes, append the index to the existing array
        driver_indices[driver].append(i)
    else:
        # If not, create a new array with the current index
        driver_indices[driver] = [i]

print("driver_indices", driver_indices)
print("len(driver_indices)", len(driver_indices))

len(max_value_index) 100
len(max_value) 100
driver_indices {'I': [0, 4, 5, 7, 16, 18, 23, 24, 50, 58, 66, 72, 81, 99], 'C': [1, 6, 10, 11, 17, 52, 57, 60, 70, 76, 77, 78, 80, 83, 87, 96, 98], 'A': [2, 13, 14, 25, 37, 38, 71, 93], 'F': [3, 8, 9, 31, 33, 36, 56, 59, 84], 'G': [12, 22, 29, 39, 46, 54, 65, 89], 'E': [15, 49, 55, 73, 79, 86, 88], 'J': [19, 20, 21, 30, 32, 43, 61, 63, 68, 82, 91, 97], 'D': [26, 40, 44, 53, 64, 69, 74, 90, 92], 'H': [27, 28, 34, 35, 42, 45, 67], 'B': [41, 47, 48, 51, 62, 75, 85, 94, 95]}
len(driver_indices) 10


In [199]:
# Create a dictionary of all drivers' routes
drivers_routes = {}

for driver in uniqueDrivers:
    print("driver", driver)

    driver_standard_index = np.array(max_value_index[driver_indices[driver]])
    driver_max_value = np.array(max_value[driver_indices[driver]])

    # Assuming driver_standard_index is a NumPy array
    unique_values_index = np.unique(driver_standard_index[~np.isnan(driver_standard_index)]).astype(int)
    # print("unique_values_index", unique_values_index)

    # Calculate the mean for each unique value
    # means = [np.mean(driver_max_value[driver_standard_index == idx]) for idx in unique_values_index if idx != np.nan]
    # print("means", means)

    weighted_sums = [np.mean(driver_max_value[driver_standard_index == idx]) * np.count_nonzero(driver_standard_index == idx) for idx in unique_values_index if idx!=-1]
    # print("weighted_sums", weighted_sums)

    best_route_Ids = []
    # Print the results for each driver
    for idx, mean in zip(unique_values_index, weighted_sums):
        # print(f"Driver: {driver}, Unique Value: {idx}, Mean: {mean}")
        best_route_Ids.append([standardIds[idx], mean])

    # Sort the routes by their mean
    best_route_Ids.sort(key=lambda x: x[1], reverse=True)
    # print("best_route_Ids", best_route_Ids)
    
    # Keep the top 5 routes
    top_5_routes = best_route_Ids[:5]

    # Update the driver's routes in the dictionary
    drivers_routes[driver] = {'driver': driver, 'routes': [id for id,value in top_5_routes]}

# Convert the dictionary to a list for JSON serialization
result_list = list(drivers_routes.values())

# Write the result to driver.json
with open(os.path.join('results', 'driver.json'), 'w') as outfile:
    json.dump(result_list, outfile, ensure_ascii=False ,indent=2)

print("JSON driver data has been written to results/driver.json")

driver A
driver B
driver C
driver D
driver E
driver F
driver G
driver H
driver I
driver J
JSON driver data has been written to results/driver.json


## STANDARD ROUTES x STANDARD ROUTES

In [200]:
x+=lol

NameError: name 'x' is not defined

In [ ]:
@njit(cache=True, nogil=True, parallel=True)
def compute_subset_similarity_matrix_and_merch(matrix, matrixMerch, progress_proxy):
    n = matrix.shape[0]
    n1 = matrix.shape[1]
    m = matrixMerch.shape[1]
    similarity_pairs = np.zeros((n,n))
    subset2 = matrix
    subset2Merch = matrixMerch
    squareMatrix = np.full((n, m), 2)
    routeWeights = np.full(n, 0.9)
    merchWeights = np.full(n, 0.1)
    print("n", n, "m", m)
    print("matrix merch", matrixMerch.shape)
    print("matrix square", squareMatrix.shape)
    print("routeWeights", routeWeights.shape)
    print("merchWeights", merchWeights.shape)
    normsSubset2Merch = np.sqrt(np.sum(np.power(subset2Merch, squareMatrix), axis=1))
    for i in prange(n):
        subset1 = matrix[i].reshape(1, -1) #replicate_row(subset_matrix, i)  
        subset1Merch = matrixMerch[i].reshape(1, -1)
        #print("subset1", subset1.shape)
        #print("subset2", subset2.shape)
        
        min_matrix = np.minimum(subset1, subset2)
        sum_min_matrix = np.sum(min_matrix, axis=-1)
        
        max_matrix = np.maximum(subset1, subset2)
        sum_max_matrix = np.sum(max_matrix, axis=-1)
        
        #print("sum_min_matrix", sum_min_matrix.shape)
        
        #print("merch1", subset1Merch.shape)
        #print("merch2", subset2Merch.shape)
        
        #distMerch = 1 - np.abs(np.dot(subset1Merch, subset2Merch.T) / (np.linalg.norm(subset1Merch) * np.linalg.norm(subset2Merch)))
        #distMerch = 1 - (((subset1Merch * subset2Merch).sum(axis=1) / (np.sqrt(np.sum(np.power(subset1Merch, squareMatrix),axis=1)) * normsSubset2Merch)) + 1) / 2
        
        
        min_matrix_merch = np.minimum(subset1Merch, subset2Merch)
        sum_min_matrix_merch = np.sum(min_matrix_merch, axis=-1)
        
        max_matrixMerch = np.maximum(subset1Merch, subset2Merch)
        sum_max_matrixMerch = np.sum(max_matrixMerch, axis=-1)
        distMerch = 1 - (sum_min_matrix_merch / sum_max_matrixMerch)
        
        # if i == 0 or i == n-1:
        #     print("i", i, "distMerch", distMerch.shape, distMerch)
        #     print("i", i, "sum_min_matrix", (1 - (sum_min_matrix / sum_max_matrix)).shape, (1 - (sum_min_matrix / sum_max_matrix)))
        #     print("i", i, "prod", ((1 - (sum_min_matrix / sum_max_matrix)) * distMerch).shape, ((1 - (sum_min_matrix / sum_max_matrix)) * distMerch))
        #print(i, (1 - (sum_min_matrix / sum_max_matrix)) * distMerch)
        #similarity_pairs[i] = ((1 - (sum_min_matrix / sum_max_matrix)) + distMerch)/2
        routeDistance = 1 - (sum_min_matrix / sum_max_matrix)
        similarity_pairs[i] = np.power(routeDistance, routeWeights) * np.power(distMerch, merchWeights)
        # if np.isnan(similarity_pairs[i]).any():
        #     np.set_printoptions(threshold=10000)
        #     # print("similarity_pairs[i]", similarity_pairs[i])
        #     print("dist merch", distMerch)
        #     # print("dist routes", routeDistance)
        #     # print("pow1", np.power(routeDistance, routeWeights))
        #     # print("pow2", np.power(distMerch, merchWeights))
        #     # print("prod", np.power(routeDistance, routeWeights) * np.power(distMerch, merchWeights))
        #     # print("powers", routeWeights, merchWeights)
        #     print("BROKEN")
        #     return
        #similarity_pairs[i] = (1 - (sum_min_matrix / sum_max_matrix)) * (distMerch)
        #similarity_pairs[i] = 1 - (sum_min_matrix / sum_max_matrix + distMerch) / 2
        # if similarity_pairs[i] >= 1:
        #     print("similarity_pairs[i]", similarity_pairs[i])
        #     print("dist merch", distMerch, "cosine ", np.abs(np.dot(subset1Merch, subset2Merch) / (np.linalg.norm(subset1Merch) * np.linalg.norm(subset2Merch))))
        #     print("dist routes", (1 - (sum_min_matrix / sum_max_matrix)))
        #     print("prod", (1 - (sum_min_matrix / sum_max_matrix)) * distMerch)
        progress_proxy.update(1)
    return similarity_pairs

In [ ]:
def jaccard_similarity_minhash_lsh_route_merch(matrix, matrixMerch, thresh_user=0.2):
    #similarity_matrix = csr_matrix((matrix.shape[0], matrix.shape[0]), dtype=np.float64)
    #similarity_matrix = lil_matrix((matrix.shape[0], matrix.shape[0]), dtype=np.float64)
    pairs = lsh(matrix, thresh_user=thresh_user)
    #uniqueRows = np.unique([i for i, j in pairs] + [j for i, j in pairs])
    uniqueRowsSet = set([i for i, j in pairs] + [j for i, j in pairs]) # (1,2) (1,4) (1,5)
    neverSeen = set([i for i in range(matrix.shape[0])]) - uniqueRowsSet
    print("neverSeen", neverSeen)
    #print("uniqueRows numpy", len(uniqueRows))
    print("num of subset of rows to check similarity:", len(uniqueRowsSet))
    #print(" num of pairs", len(uniqueRowsSet)*(len(uniqueRowsSet)-1)/2)
    print(" num of pairs", len(pairs))
    print(" instead of", matrix.shape[0]*(matrix.shape[0]-1)/2)
    print("improved by", (1 - len(pairs) / (matrix.shape[0]*(matrix.shape[0]-1)/2)) *100, "%")
    #print("num of pairs", len(pairs))
  
        
    print("Computing jaccard similarity on subset matrix...")
    
    sortedUniqueRowsSet = sorted(list(uniqueRowsSet))
    subset_matrix = matrix[sortedUniqueRowsSet]
    subset_matrixMerch = matrixMerch[sortedUniqueRowsSet]
    print("subset_matrix", subset_matrix.shape, subset_matrix[0])
    print("subset_matrixMerch", subset_matrixMerch.shape, subset_matrixMerch[0])
    with ProgressBar(total=len(sortedUniqueRowsSet)) as progress:
        subset_sim_matrix = compute_subset_similarity_matrix_and_merch(subset_matrix, subset_matrixMerch, progress)
    print("subset_sim_matrix", subset_sim_matrix.shape, subset_sim_matrix[0])
    print("subset_sim_matrix contains nan", np.isnan(subset_sim_matrix).any())
    print("nan indices", len(np.argwhere(np.isnan(subset_sim_matrix))), np.argwhere(np.isnan(subset_sim_matrix)))
    
 
    print("Mapping back to original matrix...")
    
    lenMatrixNoNeverSeen = matrix.shape[0] - len(neverSeen)
    
    # remove never seen rows and map indices
    map_indices = {}
    sortedNeverSeen = sorted(list(neverSeen))
    counter = 0
    for i in range(matrix.shape[0]):
        if i in sortedNeverSeen:
            continue
        map_indices[i] = counter
        counter += 1
        
    print("map_indices", map_indices)
    map_indices_back = {v: k for k, v in map_indices.items()}
    
    
    #similarity_matrix.setdiag(1)
    subset_sim_matrix = csr_matrix(subset_sim_matrix)
    
    return subset_sim_matrix, map_indices_back


In [ ]:
def jaccard_similarity_matrix_merch(matrix):
    print("matrix", matrix.shape)
    min_matrix = np.minimum(matrix[:, None, :], matrix[None, :, :]) # (10, 100) -> (10, 1, 100) -> (1, 10, 100) -> (10, 10, 100)
    sum_min_matrix = np.sum(min_matrix, axis=-1)
    print("sum_min_matrix", sum_min_matrix.shape)
    
    max_matrix = np.maximum(matrix[:, None, :], matrix[None, :, :])
    sum_max_matrix = np.sum(max_matrix, axis=-1)
    print("sum_max_matrix", sum_max_matrix.shape)
    
    jaccard_similarity = sum_min_matrix / sum_max_matrix
    return jaccard_similarity

In [ ]:
# convert routes and merchandise to binary matrices
# binary matrix where each row represents a route
print("Creating route binary matrix...")
route_matrix, route_matrix_standard = create_binary_matrices(actualSets, standardSets)
print("\nroute_matrix standard", route_matrix_standard.shape, route_matrix_standard[0])

print("Minhashing route matrix...")    
num_hash_functions = find_num_hashes_minhash(route_matrix_standard)
route_matrix_standard = minhash(route_matrix_standard, num_hash_functions if num_hash_functions % 2 == 0 else num_hash_functions + 1)
print("\nroute_matrix minhash", route_matrix_standard.shape, route_matrix_standard[0])
# binary matrix where each row represents merchandise

print("Creating merchandise binary matrix...")
merch_matrix = np.array([s[2] for s in standardSets])

print("\nmerch_matrix", merch_matrix.shape, merch_matrix)
print("merch_matrix contains nan", np.isnan(merch_matrix).any())


print("Computing Jaccard similarity route matrix...")
actualSetsDistances, map_indices_back = jaccard_similarity_minhash_lsh_route_merch(route_matrix_standard, merch_matrix, thresh_user=0.4)
#route_similarity = jaccard_similarity_matrix(route_matrix)
print("\nactualSetsDistances", type(actualSetsDistances), actualSetsDistances.shape,actualSetsDistances[0, 0], actualSetsDistances[0])
print("map indices back", map_indices_back)

Creating route binary matrix...

route_matrix standard (10, 3098) [0. 0. 0. ... 0. 0. 0.]
Minhashing route matrix...


minhashing: 100%|██████████| 10/10 [00:00<00:00, 1254.43it/s]



route_matrix minhash (10, 150) [162. 209.   4.  60.  89.  58. 248.   1.  85. 490.  55.  66. 156. 560.
  18.  33. 168.   9.  57.  33.  77.  44.  90. 365.   2.  52.  49. 224.
  11.  18. 471. 136.  29. 239.  84.  22. 127. 146. 128. 117.   6.   6.
 180.  61.  68.  42.   1.  42. 145.   4. 279. 206.  74. 309.  29. 181.
  16. 116.  16. 183.   2. 239. 267.  57. 108. 337.  41. 271.  67.  12.
  14.  70. 205. 129. 221. 143.  35. 208.  58.  46.  51. 101. 121.  66.
  70. 198.  50.  48.  61. 374.  17.  34. 177. 122.  30. 119.  22.  26.
 100.  44.   8.  50. 147.  46. 163. 152.  29.  34.  59.  18. 132.  60.
 215.  99.  65. 102.  15.  35.  69.  92.  65.  36. 416.  44.  58. 166.
 140.  71. 535. 248.  93.  20.  71. 124.  70. 158. 137.  71. 118. 166.
   6. 100.  23. 188. 202.  94. 173.  26. 112.  44.]
Creating merchandise binary matrix...

merch_matrix (10, 50) [[0.24172414 0.34586207 0.24965517 0.28275862 0.33482759 0.21206897
  0.19551724 0.20827586 0.26206897 0.22586207 0.23413793 0.22827586
  0.21758

100%|██████████| 10/10 [00:00<00:00, 10027.02it/s]

neverSeen {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
num of subset of rows to check similarity: 0
 num of pairs 0
 instead of 45.0
improved by 100.0 %
Computing jaccard similarity on subset matrix...


IndexError: index 0 is out of bounds for axis 0 with size 0